In [1]:
import numpy as np
import matplotlib.pyplot as plt
import mltools as ml


X = np.genfromtxt("data/X_train.txt")
Y = np.genfromtxt("data/Y_train.txt")

Xt, Xv, Yt, Yv = ml.splitData(X, Y, 0.80)

Xe = np.genfromtxt('data/X_test.txt')

In [2]:
nClust = 15
np.random.seed(0)
Z,T,pZ,ll = ml.cluster.gmmEM(Xt[:10000,:],K=nClust,init='k++',max_iter=50)

Cluster_GMM = ml.bayes.gaussClassify()
# Manually copy the EM Gaussian components into the Gaussian Bayes Classifier:
Cluster_GMM.classes = np.arange(nClust)
Cluster_GMM.means = T['mu']
Cluster_GMM.covars = [ T['sig'][:,:,i]+.05*np.eye(Xt.shape[1]) for i in range(nClust) ]
Cluster_GMM.probs = T['pi']

# Find cluster membership probabilities for each data set:
XtC = Cluster_GMM.predictSoft(Xt)
XvC = Cluster_GMM.predictSoft(Xv)
XeC = Cluster_GMM.predictSoft(Xe)

# Create extended feature set:  features X times membership probability for each cluster
XtC2 = np.einsum('ij,ik->ijk',XtC,Xt).reshape((Xt.shape[0],Xt.shape[1]*nClust))
XvC2 = np.einsum('ij,ik->ijk',XvC,Xv).reshape((Xv.shape[0],Xv.shape[1]*nClust))
XeC2 = np.einsum('ij,ik->ijk',XeC,Xe).reshape((Xe.shape[0],Xe.shape[1]*nClust))

# Regress (should really use a classifier...)
linr2 = ml.linear.linearRegress(XtC2,Yt, reg=1e-3)
Pv3 = linr2.predict(XvC2)[:,0]
Pe3 = linr2.predict(XeC2)[:,0]

#toKaggle('Pe3.csv',Pe3)
print "3: Clustered LinRegress: MSE ~",linr2.mse(XvC2,Yv),'; AUC = ',auc(Pv3,Yv)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices